In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import os

# Load pre-trained model
model = models.resnet50(pretrained=True)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load image
def load_image(img_path):
    try:
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0)
        return img, img_tensor
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None, None

In [6]:
def get_cam(model, img_tensor):
    # Get the last convolutional layer
    final_conv_layer = model.layer4[-1].conv2
    feature_map = None
    def hook_function(module, input, output):
        nonlocal feature_map
        feature_map = output
    hook = final_conv_layer.register_forward_hook(hook_function)
    
    # Forward pass through the model
    with torch.no_grad():
        output = model(img_tensor)
    hook.remove()
    
    # Get the weights of the final fully connected layer
    params = list(model.parameters())
    weight_softmax = params[-2]
    
    # Get the class with the highest score
    _, class_idx = torch.max(output, 1)
    
    # Get the CAM
    cam = torch.zeros(feature_map.shape[2:])
    for i, w in enumerate(weight_softmax[class_idx]):
        cam += w * feature_map[0, i, :, :]
    cam = cam.cpu().numpy()
    cam = cam - cam.min()
    cam = cam / cam.max()
    return cam

def visualize_cam(img, cam):
    plt.imshow(img)
    plt.imshow(cam, cmap='jet', alpha=0.5)
    plt.axis('off')
    plt.show()


In [8]:
# Define the paths to your dataset
positive_folder = r'D:\Weapon Detection Project DRDO\KnivesImagesDatabase\POSITIVES_ALL'
negative_folder = r'D:\Weapon Detection Project DRDO\KnivesImagesDatabase\NEGATIVES_ALL'

# Function to detect knives
def detect_knives(image_path):
    img, img_tensor = load_image(image_path)
    if img_tensor is None:
        print(f"Skipping {image_path}")
        return
    cam = get_cam(model, img_tensor)
    visualize_cam(img, cam)

# Run inference on positive images
positive_images = os.listdir(positive_folder)
for img_name in positive_images[:5]:  # Just to visualize a few examples
    img_path = os.path.join(positive_folder, img_name)
    detect_knives(img_path)

# Run inference on negative images
negative_images = os.listdir(negative_folder)
for img_name in negative_images[:5]:  # Just to visualize a few examples
    img_path = os.path.join(negative_folder, img_name)
    detect_knives(img_path)

RuntimeError: The size of tensor a (2048) must match the size of tensor b (7) at non-singleton dimension 1